# Model Training

Prepare dataframe 

In [1]:
from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from gensim.models import Word2Vec
from sklearn.svm import LinearSVC
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from utils.text_preprocess import LemmaTokenizer
import nltk
import numpy as np
from nltk.tokenize import sent_tokenize
from sklearn.experimental import enable_halving_search_cv

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('punkt_tab')
        
df = pd.read_csv('data/amazon_reviews.txt', sep='\t')
df['VERIFIED_PURCHASE'] = df['VERIFIED_PURCHASE'].map({'Y': 1, 'N': 0})

word2vec_model_file = 'models/word2vec_100_v2.model'
model = Word2Vec.load(word2vec_model_file)

def average_word_embeddings(sentence, model):
    words = LemmaTokenizer()(sentence)
    word_vectors = [model.wv[word] for word in words if word in model.wv]
    
    if len(word_vectors) == 0:
        return np.zeros(model.vector_size)
    
    return np.mean(word_vectors, axis=0)

# Define a function to process each review
def process_review(review_text, model):
    sentences = sent_tokenize(review_text)
    sentence_embeddings = [average_word_embeddings(sentence, model) for sentence in sentences]
    return sentence_embeddings

df['Sentence_Embeddings'] = df['REVIEW_TEXT'].apply(lambda x: process_review(x, model))


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Utente\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Utente\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Utente\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
E:\PycharmProjects\amazon_fake_reviews_detection\utils\text_preprocess.py:28: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  text = BeautifulSoup(text, "html.parser").get_text()


Create pipelines

In [2]:
from utils.transformers import *
    
classifiers = [
    CalibratedClassifierCV(LinearSVC(random_state=42, dual="auto")),
    RandomForestClassifier(random_state=42, n_jobs=-1),
]
vectorizers = [
    TfidfVectorizer(tokenizer=LemmaTokenizer(), token_pattern=None),
    CountVectorizer(tokenizer=LemmaTokenizer(), token_pattern=None),
]

pipes = []
pipes_names = []
for clf in classifiers:
    for vectorizer in vectorizers:
        for useEmbeddings in [True]:
            for useVP in [True, False]:
                transformers = [
                    ('vectorizer', vectorizer, 'REVIEW_TEXT')
                ]
                if useEmbeddings:
                    transformers.append(('embeddings', SentenceEmbeddingTransformer(), 'Sentence_Embeddings'))
                if useVP:
                    transformers.append(('vp', 'passthrough', ['VERIFIED_PURCHASE']))
                
                # No need for DenseTransformer for other classifiers
                pipes.append(Pipeline([
                    ('preprocessor', ColumnTransformer(transformers, n_jobs=-1)),
                    ('clf', clf)
                ]))
                    
                pipes_names.append(f'{clf.__class__.__name__}_'
                                   f'{vectorizer.__class__.__name__}'
                                   f'{"_WE" if useEmbeddings else ""}'
                                   f'{"_VP" if useVP else ""}')

Nested cross-validation

In [3]:
from nltk import sent_tokenize
import pandas as pd
import joblib
from sklearn.model_selection import HalvingGridSearchCV, StratifiedKFold
from utils.text_preprocess import LemmaTokenizer


param_grids = {
    'RandomForestClassifier': {
        'clf__n_estimators': [800, 1000, 1200],
        'clf__criterion': ['entropy', 'log_loss'],
        'clf__max_depth': [None, 20, 40, 60],
        'clf__bootstrap': [False, True],
    },
    'CalibratedClassifierCV': {
        'clf__estimator__C': [0.5, 1, 1.5],
        'clf__estimator__max_iter': [2000, 4000],
        'clf__estimator__loss': ['hinge', 'squared_hinge'],
    },
}

inner_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
outer_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
results = {}
best_accuracy = 0
best_model = None

for pipe, pipe_name in zip(pipes, pipes_names):
    print(pipe_name)
    clf_name = pipe.steps[-1][1].__class__.__name__
    grid = param_grids[clf_name]
    clf = HalvingGridSearchCV(pipe, grid, cv=inner_cv, factor=3, random_state=42, n_jobs=-1)
    results[pipe_name] = [[], [], [], []]
    
    # Loop instead of cross_val_score to have more control over printing
    for fold, (train_idx, test_idx) in enumerate(outer_cv.split(df, df['LABEL'])):
        print(f"Fold {fold}:")
        
        # Split the data
        X_train, X_test = df.iloc[train_idx], df.iloc[test_idx]
        y_train, y_test = df['LABEL'].iloc[train_idx], df['LABEL'].iloc[test_idx]
        
        clf.fit(X_train, y_train)
        y_pred = clf.predict(X_test)
        results[pipe_name][0].append(accuracy_score(y_test, y_pred))
        results[pipe_name][1].append(f1_score(y_test, y_pred, pos_label='__label1__'))
        results[pipe_name][2].append(roc_auc_score(y_test, clf.predict_proba(X_test)[:,1]))
        results[pipe_name][3].append(clf.best_params_)
        
        # Update best model 
        if results[pipe_name][0][-1] > best_accuracy:
            best_accuracy = results[pipe_name][0][-1]
            best_model = clf.best_estimator_

        print(f"Best params: {clf.best_params_}\n"
              f"Test accuracy: { results[pipe_name][0][-1]}\n"
              f"Test F1: { results[pipe_name][1][-1]}\n"
              f"Test AUC: { results[pipe_name][2][-1]}\n")
    
    print(f'Mean Accuracy: {np.mean(results[pipe_name][0])}\n'
          f'Mean F1: {np.mean(results[pipe_name][1])}\n'
          f'Mean AUC: {np.mean(results[pipe_name][2])}\n')

# save results to csv file
results_str = {key: [str(value) for value in values] for key, values in results.items()}

df = pd.DataFrame()
for key, value in results_str.items():
    temp_df = pd.DataFrame([value], columns=['accuracy', 'f1_score', 'AUC', 'hyperparameters'])
    temp_df.insert(0, 'model', key)
    df = pd.concat([df, temp_df], ignore_index=True)
df.to_csv('results2.csv', index=False)

# save best model
joblib.dump(best_model, 'models/best_model_2.pkl')

CalibratedClassifierCV_TfidfVectorizer_WE_VP
Fold 0:
Best params: {'clf__estimator__C': 1, 'clf__estimator__loss': 'hinge', 'clf__estimator__max_iter': 2000}
Test accuracy: 0.7985714285714286
Test F1: 0.7971223021582734
Test AUC: 0.8566149659863946

Fold 1:
Best params: {'clf__estimator__C': 1, 'clf__estimator__loss': 'hinge', 'clf__estimator__max_iter': 2000}
Test accuracy: 0.7985714285714286
Test F1: 0.7943607194944093
Test AUC: 0.863936507936508

Fold 2:


C:\Users\Utente\anaconda3\envs\DMML_project\Lib\site-packages\sklearn\svm\_base.py:1235: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Best params: {'clf__estimator__C': 1, 'clf__estimator__loss': 'hinge', 'clf__estimator__max_iter': 2000}
Test accuracy: 0.8080952380952381
Test F1: 0.8055957549445248
Test AUC: 0.8621886621315193

Fold 3:
Best params: {'clf__estimator__C': 1, 'clf__estimator__loss': 'hinge', 'clf__estimator__max_iter': 2000}
Test accuracy: 0.7980952380952381
Test F1: 0.7947725072604066
Test AUC: 0.8575210884353742

Fold 4:


C:\Users\Utente\anaconda3\envs\DMML_project\Lib\site-packages\sklearn\svm\_base.py:1235: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Best params: {'clf__estimator__C': 1, 'clf__estimator__loss': 'hinge', 'clf__estimator__max_iter': 2000}
Test accuracy: 0.8052380952380952
Test F1: 0.8023199613339778
Test AUC: 0.8658049886621315

Fold 5:
Best params: {'clf__estimator__C': 1, 'clf__estimator__loss': 'hinge', 'clf__estimator__max_iter': 2000}
Test accuracy: 0.8061904761904762
Test F1: 0.8005879470847623
Test AUC: 0.8754766439909296

Fold 6:
Best params: {'clf__estimator__C': 0.5, 'clf__estimator__loss': 'squared_hinge', 'clf__estimator__max_iter': 2000}
Test accuracy: 0.8033333333333333
Test F1: 0.7998061076102763
Test AUC: 0.8659555555555555

Fold 7:


...

C:\Users\Utente\anaconda3\envs\DMML_project\Lib\site-packages\joblib\externals\loky\process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Best params: {'clf__bootstrap': False, 'clf__criterion': 'log_loss', 'clf__max_depth': None, 'clf__n_estimators': 800}
Test accuracy: 0.63
Test F1: 0.65015758667267
Test AUC: 0.6764371882086169

Fold 9:
Best params: {'clf__bootstrap': True, 'clf__criterion': 'entropy', 'clf__max_depth': 60, 'clf__n_estimators': 800}
Test accuracy: 0.6157142857142858
Test F1: 0.630325240494732
Test AUC: 0.6646648526077098

Mean Accuracy: 0.6351428571428571
Mean F1: 0.6513258430756702
Mean AUC: 0.6871772335600907

RandomForestClassifier_CountVectorizer_VP
Fold 0:
Best params: {'clf__bootstrap': False, 'clf__criterion': 'log_loss', 'clf__max_depth': 60, 'clf__n_estimators': 1000}
Test accuracy: 0.8042857142857143
Test F1: 0.7925290257445734
Test AUC: 0.8699990929705216

Fold 1:
Best params: {'clf__bootstrap': True, 'clf__criterion': 'log_loss', 'clf__max_depth': None, 'clf__n_estimators': 1000}
Test accuracy: 0.7980952380952381
Test F1: 0.7838939857288482
Test AUC: 0.8599283446712017

Fold 2:
Best params:

C:\Users\Utente\anaconda3\envs\DMML_project\Lib\site-packages\joblib\externals\loky\process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Best params: {'clf__bootstrap': False, 'clf__criterion': 'entropy', 'clf__max_depth': None, 'clf__n_estimators': 800}
Test accuracy: 0.8095238095238095
Test F1: 0.8005982053838484
Test AUC: 0.8630893424036281

Mean Accuracy: 0.8053333333333335
Mean F1: 0.7921503485299014
Mean AUC: 0.8710356916099773

RandomForestClassifier_CountVectorizer
Fold 0:
Best params: {'clf__bootstrap': True, 'clf__criterion': 'log_loss', 'clf__max_depth': 40, 'clf__n_estimators': 800}
Test accuracy: 0.6247619047619047
Test F1: 0.6552930883639545
Test AUC: 0.6805124716553288

Fold 1:
Best params: {'clf__bootstrap': True, 'clf__criterion': 'log_loss', 'clf__max_depth': 60, 'clf__n_estimators': 800}
Test accuracy: 0.6466666666666666
Test F1: 0.6734154929577465
Test AUC: 0.7107437641723356

Fold 2:
Best params: {'clf__bootstrap': True, 'clf__criterion': 'entropy', 'clf__max_depth': 40, 'clf__n_estimators': 1200}
Test accuracy: 0.6142857142857143
Test F1: 0.6481320590790617
Test AUC: 0.682477097505669

Fold 3:
Best

['models/best_model.pkl']

## Statistical comparison

Test normality

In [ ]:
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
import seaborn as sns
import matplotlib.pyplot as plt

model1 = Pipeline([
    ('preprocessor', ColumnTransformer([
        ('vectorizer', TfidfVectorizer(tokenizer=LemmaTokenizer(), token_pattern=None), 'REVIEW_TEXT'),
        ('vp', 'passthrough', ['VERIFIED_PURCHASE']),
        ('embeddings', SentenceEmbeddingTransformer(), 'Sentence_Embeddings'),
    ], n_jobs=-1)),
    ('clf', RandomForestClassifier(random_state=42, n_jobs=-1)),
])
model2 = Pipeline([
    ('preprocessor', ColumnTransformer([
        ('vectorizer', TfidfVectorizer(tokenizer=LemmaTokenizer(), token_pattern=None), 'REVIEW_TEXT'),
        ('vp', 'passthrough', ['VERIFIED_PURCHASE']),
        ('embeddings', SentenceEmbeddingTransformer(), 'Sentence_Embeddings'),
    ], n_jobs=-1)),
    ('clf', LinearSVC(random_state=42, dual="auto")),
])

rkf = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=42)

# Store the differences between model performances
differences = []

for train_index, test_index in rkf.split(df, df['LABEL']):
    # Split the data
    X_train, X_test = df.iloc[train_index], df.iloc[test_index]
    y_train, y_test = df['LABEL'].iloc[train_index], df['LABEL'].iloc[test_index]

    model1.fit(X_train, y_train)
    model2.fit(X_train, y_train)
    
    y_pred1 = model1.predict(X_test)
    y_pred2 = model2.predict(X_test)
    
    acc1 = accuracy_score(y_test, y_pred1)
    acc2 = accuracy_score(y_test, y_pred2)
    
    differences.append(acc1 - acc2)

differences = np.array(differences)

# Histogram
sns.histplot(differences, kde=True)
plt.show()

# Q-Q plot
from scipy import stats
stats.probplot(differences, dist="norm", plot=plt)
plt.show()

Perform Wilcoxon signed-rank test

In [ ]:
from scipy.stats import wilcoxon

w_p_value = wilcoxon(differences)
print('Wilcoxon signed-rank test p-value:', w_p_value)